##### requests 라이브러리

In [2]:
import requests

page = requests.get("https://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [3]:
# status_code 의미
# 2xx : 성공
# 4xx : 클라이언트 에러
# 5xx : 서버 에러
page.status_code

200

In [4]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

##### BeautifulSoup 라이브러리

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [7]:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [8]:
# Doctype : 문서 타입에 대한 정보
# NavigableString : HTML 문서에서 찾아낸 텍스트
# Tag : 중첩(nested) 태그
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [9]:
html = list(soup.children)[2]
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

In [10]:
# body 태그 안 자식에 대한 결과
body = list(html.children)[3]
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [11]:
# p 태그 안 텍스트
p = list(body.children)[1]
p.get_text()

'Here is some simple content for this page.'

##### 태그명으로 태그 검색

In [12]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [13]:
# find_all은 list를 리턴함.
# 따라서 첫 번쨰 텍스트를 얻기 위해 다음과 같이 함.
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [14]:
# 첫 번째 인스턴스만 찾기 원할 경우 find 이용
soup.find('p')

<p>Here is some simple content for this page.</p>

##### class와 id로 태그 검색

In [15]:
page = requests.get("https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [16]:
soup.find_all('p', class_ = 'outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [17]:
soup.find_all(class_ = "outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

##### CSS 선택자

In [18]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>]

##### 날씨 데이터 스크래핑

In [19]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id = "seven-day-forecast")
forecast_items = seven_day.find_all(class_ = "tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly clear, with a low around 48. West wind 12 to 17 mph becoming light southwest. Winds could gust as high as 22 mph. " class="forecast-icon" src="newimages/medium/nfew.png" title="Tonight: Mostly clear, with a low around 48. West wind 12 to 17 mph becoming light southwest. Winds could gust as high as 22 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 48 °F
 </p>
</div>


In [20]:
period = tonight.find(class_ = "period-name").get_text()
short_desc = tonight.find(class_ = "short-desc").get_text()
temp = tonight.find(class_ = "temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Mostly Clear
Low: 48 °F


In [21]:
img = tonight.find("img")
desc = img['title']
print(desc)

Tonight: Mostly clear, with a low around 48. West wind 12 to 17 mph becoming light southwest. Winds could gust as high as 22 mph. 


##### 모든 정보 추출하기

In [22]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight']

In [24]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunnythen MostlySunny andBreezy', 'Mostly Clearand Breezythen MostlyClear', 'Mostly Sunny', 'Mostly Clear']
['Low: 48 °F', 'High: 66 °F', 'Low: 48 °F', 'High: 68 °F', 'Low: 49 °F', 'High: 63 °F', 'Low: 50 °F', 'High: 66 °F', 'Low: 50 °F']
['Tonight: Mostly clear, with a low around 48. West wind 12 to 17 mph becoming light southwest. Winds could gust as high as 22 mph. ', 'Friday: Sunny, with a high near 66. West southwest wind 9 to 14 mph increasing to 15 to 20 mph in the afternoon. Winds could gust as high as 25 mph. ', 'Friday Night: Mostly clear, with a low around 48. West wind 12 to 17 mph becoming light and variable  after midnight. Winds could gust as high as 23 mph. ', 'Saturday: Mostly sunny, with a high near 68. Calm wind becoming northwest 5 to 9 mph in the afternoon. Winds could gust as high as 24 mph. ', 'Saturday Night: Partly cloudy, with a low around 49. West southwest wind 11 to 16 mph, w

##### 모든 데이터를 Pandas Dataframe으로 합치기

In [26]:
import pandas as pd
weather = pd.DataFrame({
    "period" : period,
    "short_desc" : short_descs,
    "temp" : temps,
    "desc" : descs
})
weather

,period,short_desc,temp,desc
0,Tonight,Mostly Clear,Low: 48 °F,"Tonight: Mostly clear, with a low around 48. W..."
1,Tonight,Sunny,High: 66 °F,"Friday: Sunny, with a high near 66. West south..."
2,Tonight,Mostly Clear,Low: 48 °F,"Friday Night: Mostly clear, with a low around ..."
3,Tonight,Mostly Sunny,High: 68 °F,"Saturday: Mostly sunny, with a high near 68. C..."
4,Tonight,Partly Cloudy,Low: 49 °F,"Saturday Night: Partly cloudy, with a low arou..."
5,Tonight,Mostly Sunnythen MostlySunny andBreezy,High: 63 °F,"Sunday: Mostly sunny, with a high near 63. Bre..."
6,Tonight,Mostly Clearand Breezythen MostlyClear,Low: 50 °F,"Sunday Night: Mostly clear, with a low around ..."
7,Tonight,Mostly Sunny,High: 66 °F,"Monday: Mostly sunny, with a high near 66."
8,Tonight,Mostly Clear,Low: 50 °F,"Monday Night: Mostly clear, with a low around 50."


In [27]:
temp_nums = weather["temp"].str.extract(r'(?P<digit>\d+)', expand = False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    48
1    66
2    48
3    68
4    49
5    63
6    50
7    66
8    50
Name: digit, dtype: object

In [28]:
weather["temp_num"].mean()

56.44444444444444

In [29]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [30]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
0,Tonight,Mostly Clear,Low: 48 °F,"Tonight: Mostly clear, with a low around 48. W...",48,True
2,Tonight,Mostly Clear,Low: 48 °F,"Friday Night: Mostly clear, with a low around ...",48,True
4,Tonight,Partly Cloudy,Low: 49 °F,"Saturday Night: Partly cloudy, with a low arou...",49,True
6,Tonight,Mostly Clearand Breezythen MostlyClear,Low: 50 °F,"Sunday Night: Mostly clear, with a low around ...",50,True
8,Tonight,Mostly Clear,Low: 50 °F,"Monday Night: Mostly clear, with a low around 50.",50,True
